# ABOUT 
- this notebook tries experiments with a fast version of priority queue
- background:
    - MongoDB returns documents in the form of a iterable Cursor, which we will loop through
    - for each document, we compute a similarity score and we want only the topk documents
    - thus, we will use a fast priority queue to select the topk documents with the highest similarity score
- code source:
    - https://github.com/abadger/priorityqueue

In [1]:
# Copyright: (c) 2019, Toshio Kuratomi <a.badger@gmail.com>
# GNU General Public License v3.0+ (see COPYING or https://www.gnu.org/licenses/gpl-3.0.txt)
import heapq
from collections import deque

class PriorityQueue:
    def __init__(self):
        self.ordered_store = []
        self.lookup_store = {}

    def push(self, value, priority=None):
        if priority is None:
            priority = 0

        if priority not in self.lookup_store:
            heapq.heappush(self.ordered_store, priority)
            self.lookup_store[priority] = deque()
        self.lookup_store[priority].append(value)

    def pop(self):
        priority = self.ordered_store[0]
        store = self.lookup_store[priority]
        value = store.popleft()

        if not store:
            heapq.heappop(self.ordered_store)

        return value

    def __len__(self):
        total_len = 0
        for queue in self.lookup_store.values():
            total_len += len(queue)
        return total_len

    def __bool__(self):
        return True if self.ordered_store else False

    def compact(self):
        """Reduce the memory requirements of the queue"""
        for priority in list(self.lookup_store.keys()):
            if not self.lookup_store[priority]:
                del self.lookup_store[priority]
      
    
    
"""
Plus Version only requires push to be called (makes things easier)
"""
class PriorityQueuePlus(PriorityQueue):
    def __init__(self, topk, max_size):
        super().__init__()
        self.topk = topk
        self.max_size = max_size
    def push(self, value, priority=None):
        super().push(value, priority)
        if self.__len__() > self.topk:
            self.pop()
        if len(self.lookup_store)> self.max_size:
            self.compact() 
    def get_topk(self):
        self.compact()
        output = [(item[0], score) for score, item in self.lookup_store.items()]
        return sorted(output, key = lambda element: element[1], reverse = True)

## mock data
- assign letters a-z with ascending numbers which represent their priority
- shuffle the mock data
- passing the mock data through the queue should return the last k letters of the alphabets because they were assigned the high priorities

In [2]:
import numpy as np
from random import shuffle
values = list("abcdefghijklmnopqrstuvwxyz")
priorities = np.arange(0,10,10/len(values))
data = list(tuple(zip(values,priorities)))
shuffle(data)
data

[('a', 0.0),
 ('p', 5.769230769230769),
 ('o', 5.384615384615385),
 ('i', 3.076923076923077),
 ('h', 2.6923076923076925),
 ('s', 6.923076923076923),
 ('d', 1.153846153846154),
 ('t', 7.307692307692308),
 ('c', 0.7692307692307693),
 ('e', 1.5384615384615385),
 ('r', 6.538461538461539),
 ('j', 3.4615384615384617),
 ('w', 8.461538461538462),
 ('v', 8.076923076923077),
 ('m', 4.615384615384616),
 ('y', 9.230769230769232),
 ('b', 0.38461538461538464),
 ('x', 8.846153846153847),
 ('n', 5.0),
 ('q', 6.153846153846154),
 ('k', 3.8461538461538463),
 ('l', 4.230769230769231),
 ('g', 2.307692307692308),
 ('f', 1.9230769230769231),
 ('u', 7.6923076923076925),
 ('z', 9.615384615384617)]

## run queue
- since the topk values returned are z', 'y', 'v', 'x', 'w', the queue is implemented correctly

In [3]:
q = PriorityQueuePlus(5, 8)
for value, priority in data:
    q.push(value, priority = priority)

In [4]:
q.get_topk()

[('z', 9.615384615384617),
 ('y', 9.230769230769232),
 ('x', 8.846153846153847),
 ('w', 8.461538461538462),
 ('v', 8.076923076923077)]